<a href="https://colab.research.google.com/github/jasreman8/Multi-Agent-System-Projects-II/blob/main/deepeval_agent_testing_financial_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

- Learn how to use DeepEval to create test cases for evaluating LLM-based agents.
- Extract tool usage details from agent execution traces for evaluation.
- Define and apply DeepEval metrics like TaskCompletionMetric and ToolCorrectnessMetric.
- Interpret the results of an agent evaluation.

# Business Use Case

**Scenario:** A financial services company wants to provide its analysts or clients with an AI-powered research assistant. This assistant needs to quickly answer queries about public companies, such as their current stock performance, general company information, and relevant news.

**Problem:** Manually gathering this information from various sources (like Yahoo Finance, news aggregators) can be time-consuming. An AI assistant can automate this, but its reliability and accuracy are crucial.

*Notes:*
 - A news aggregator can be an API, tool, website, or application that collects and organizes news articles, blog posts, and other content from multiple sources, presenting them in one convenient location.

# Setup

!pip install -q yfinance==0.2.61 \
                langchain==0.3.24 \
                langchain-openai==0.3.14 \
                langchain-community==0.3.19 \
                langgraph==0.3.34 \
                deepeval==2.8.2

In [1]:
import os, json
import yfinance as yf

from typing import List
from deepeval import evaluate

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

from langgraph.prebuilt import create_react_agent

from deepeval.metrics import TaskCompletionMetric, ToolCorrectnessMetric
from deepeval.test_case import LLMTestCase, ToolCall

from langchain_community.tools.tavily_search import TavilySearchResults

from google.colab import userdata

In [2]:
openai_api_key = userdata.get('OPEN_API_KEY')

os.environ['OPENAI_API_KEY'] = openai_api_key # Set the environment variable for DeepEval
os.environ['OPENAI_BASE_URL'] = "https://aibe.mygreatlearning.com/openai/v1"
os.environ['TAVILY_API_KEY'] = userdata.get('tavily_search_api_key')

llm = ChatOpenAI(
    api_key=openai_api_key,
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o-mini',
    temperature=0
)

# Implementation Plan

In this notebook, we build and test an AI agent designed for financial research by following these general steps:

1.  **Create Specialized Tools:** Develop custom functions that the agent can use to perform specific tasks, such as fetching current stock prices or company details from the internet. Also, integrate a general web search tool.
2.  **Build the Agent:** Combine a powerful language model (the "brain") with the specialized tools. This agent will be able to understand user requests, decide which tool to use (if any), use the tool, and then formulate an answer based on the results. It's designed to "think" and "act" in a loop.
3.  **Prepare Test Scenarios:** Create a set of realistic questions or tasks that a user might ask the agent. For each question, also note down what an ideal interaction would look like, especially which tools the assistant *should* use.
4.  **Automate Test Case Generation:** Develop a process to run the agent with each test question and automatically record its final answer and exactly how it used its tools (which tools it called, what information it gave them, and what results it got back).
5.  **Define Success Criteria (Metrics):** Establish clear ways to measure the agent's performance. For example:
    *   How well did it complete the user's task?
    *   Did it use the correct tools for the job?
6.  **Evaluate the Agent:** Run all the test scenarios through the agent and use the success criteria to score its performance. This will provide insights into its strengths and weaknesses.

# Tool Definitions

The code block below defines a custom tool that the AI agent can use to fetch the current stock price for a given company ticker.
1.  **Tool Definition (`@tool` decorator):** The `@tool` decorator from LangChain automatically converts the Python function `get_stock_price` into a LangChain `Tool` object. This makes it easy for the agent to understand and use this function.
2.  **Function Signature (`def get_stock_price(ticker: str) -> str:`):**
    *   It takes one argument: `ticker` (a string representing the stock symbol, e.g., "AAPL").
    *   It's type-hinted to return a string (either the price information or an error message).
3.  **Docstring:** The docstring is crucial. LangChain agents use the docstring to understand what the tool does, what inputs it expects, and when to use it. A clear and descriptive docstring helps the LLM make better decisions about tool usage.
4.  **Fetching Data (`yf.Ticker(ticker)`):** It uses the `yfinance` library to get a `Ticker` object for the given symbol.
5.  **Price Retrieval Logic:**
    *   It first tries to get the closing price from the most recent day's history (`stock.history(period="1d")`).
    *   **Fallback Mechanism:** If the history is empty or doesn't contain a closing price (which can happen for various reasons, e.g., after market close, for certain types of tickers), it tries an alternative method: fetching `stock.info` (a dictionary of various data points) and looking for `currentPrice` or `regularMarketPrice`. This makes the tool more robust.
6.  **Formatting Output:** The retrieved price is formatted into a user-friendly string (e.g., "The current stock price for AAPL is $150.25.").
7.  **Error Handling (`try...except`):** If any error occurs during the process (e.g., invalid ticker, network issue, rate limiting), it catches the exception, prints an error message to the console, and returns a descriptive error message to the agent. This prevents the agent from crashing and allows it to potentially inform the user or try a different approach.

In [3]:
@tool
def get_stock_price(ticker: str) -> str:
    """
    Retrieves the current stock price for a given company ticker symbol using Yahoo Finance.
    Requires the company's valid stock ticker symbol (e.g., 'AAPL' for Apple, 'MSFT' for Microsoft).
    Returns the current price or an error message if the ticker is invalid or data is unavailable.
    """
    print(f"--- Tool Called: get_stock_price (Ticker: {ticker}) ---")
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1d") # Get the most recent day's data
        if hist.empty or 'Close' not in hist or hist['Close'].iloc[-1] is None:
             # Sometimes history is empty, try fetching info directly
             info = stock.info
             price = info.get('currentPrice') or info.get('regularMarketPrice')
             if price:
                 return f"The current stock price for {ticker} is ${price:.2f}."
             else:
                 return f"Could not retrieve current stock price for {ticker}. It might be delisted or data unavailable."
        else:
            current_price = hist['Close'].iloc[-1]
            return f"The current stock price for {ticker} is ${current_price:.2f}."
    except Exception as e:
        print(f"Error fetching stock price for {ticker}: {e}")
        return f"Failed to retrieve stock price for ticker {ticker}. Please ensure it's a valid ticker symbol. Error: {str(e)}"

 - Investment banks, fintech platforms, and research firms utilize RAG-based systems to support analysts in generating timely insights while ensuring compliance, accuracy, and reliability.

The code block below defines another custom tool, `get_company_info`, for the agent to retrieve general information about a company.
1.  **Tool Definition (`@tool`):** Similar to `get_stock_price`, this function is converted into a LangChain tool.
2.  **Function Signature and Docstring:** Takes a `ticker` string and returns a string containing company information or an error. The docstring explains its purpose and expected input to the agent.
3.  **Fetching Data (`stock.info`):** It uses `yf.Ticker(ticker).info`. The `.info` attribute of a `yfinance.Ticker` object returns a dictionary containing a wealth of information about the company.
4.  **Extracting Specific Information:**
    *   It uses the `.get('key', 'N/A')` dictionary method to safely extract specific fields like `longBusinessSummary`, `sector`, `industry`, `website`, and `shortName`. Using `'N/A'` as a default value ensures that if a key is missing, the program doesn't crash and instead uses "N/A".
5.  **Handling Non-Company Tickers:** It checks if both summary and sector are "N/A". This is a heuristic to identify cases where the ticker might not be for a standard company (e.g., an ETF or index), for which detailed company info wouldn't be available.
6.  **Formatting Output:** The extracted information is compiled into a well-formatted, readable string.
7.  **Error Handling (`try...except`):** Catches potential errors during the process and returns a user-friendly error message.

In [4]:
@tool # converts the function into a LangChain tool
def get_company_info(ticker: str) -> str:
    """
    Provides basic company information from Yahoo Finance using the stock ticker symbol.
    Information includes business summary, sector, industry, and website (if available).
    Requires a valid stock ticker symbol (e.g., 'GOOGL' for Alphabet).
    """
    print(f"--- Tool Called: get_company_info (Ticker: {ticker}) ---")
    try:
        stock = yf.Ticker(ticker)
        info = stock.info # Fetch dictionary of company info

        # Extract desired fields, handling potential missing keys gracefully
        summary = info.get('longBusinessSummary', 'N/A')# give me the value for the key 'longBusinessSummary'; if it doesn't exist, return 'N/A' instead of throwing an error.
        sector = info.get('sector', 'N/A')
        industry = info.get('industry', 'N/A')
        website = info.get('website', 'N/A')
        name = info.get('shortName', ticker) # Use short name if available

        if summary == 'N/A' and sector == 'N/A':
             return f"Could not retrieve detailed company information for ticker {ticker}. It might be an ETF, index, or invalid."

        # Format the output
        output = (
            f"Company Information for {name} ({ticker}):\n"
            f"- Sector: {sector}\n"
            f"- Industry: {industry}\n"
            f"- Website: {website}\n"
            f"- Summary: {summary}..."
        )
        return output

    except Exception as e:
        print(f"Error fetching company info for {ticker}: {e}")
        return f"Failed to retrieve company information for ticker {ticker}. Ensure it's a valid ticker. Error: {str(e)}"

In [5]:
tavily_search = TavilySearchResults(max_results=3, name="tavily_search_results")

# Agent Creation

The code block below brings together the tools and the LLM to create the agent.
1.  **Aggregate Tools:**
    *   `tools = [tavily_search, get_stock_price, get_company_info]`: A Python list called `tools` is created, containing all the tools available to the agent: the pre-built Tavily search tool and the two custom tools defined earlier (`get_stock_price`, `get_company_info`).
2.  **Create ReAct Agent:**
    *   `research_agent = create_react_agent(llm, tools)`: This is a key step using LangGraph.
        *   `create_react_agent` is a pre-built constructor that sets up a ReAct (Reasoning and Acting) agent.
        *   It takes the initialized `llm` (the "brain") and the list of `tools` as input.
        *   The `research_agent` object is now an "agent executor" - it can take user input and manage the entire reasoning and tool-use process.


In [6]:
tools = [tavily_search, get_stock_price, get_company_info]

research_agent =  create_react_agent(llm, tools)

# Example Agent Invocation

This block demonstrates how to use (invoke) the created agent with a sample query and how to inspect its response. This is a manual way to test if the agent is working as expected.
1.  **Define a Test Query:**
    *   `test_query1 = "..."`: A natural language question is defined. This query implicitly requires the agent to use multiple tools (stock price for "performing financially" and news search for "updates").
2.  **Invoke the Agent:**
    *   `response = research_agent.invoke(...)`: The agent is called with the `test_query1`.
    *   The input format `{'messages': [{'role': 'user', 'content': test_query1}]}` is standard for LangChain conversational agents. It mimics a chat history, starting with a user message.
    *   The `invoke` method runs the agent's reasoning loop. The agent will:
        *   Analyze the query.
        *   Potentially decide to use one or more tools (e.g., `get_stock_price` for "GOOGL", then `tavily_search_results` for news).
        *   Get results from these tools.
        *   Formulate a final answer.
    *   The `response` variable will contain the entire history of the interaction, including intermediate thoughts, tool calls, tool responses, and the final AI message.
3.  **Print Agent Messages:**
    *   `for message in response['messages']:`: The code iterates through all messages in the agent's response. `response['messages']` is a list of message objects (e.g., `HumanMessage`, `AIMessage`, `ToolMessage`).
    *   `message.pretty_print()`: This method, available on LangChain message objects, prints the message content in a human-readable format, often showing the role (user, AI, tool), content, and any tool call information. This helps in understanding the agent's step-by-step process.

**Test Case 1: Implied needs (Performance -> Stock Price, Updates -> News)**

In [7]:
test_query1 = "How is Alphabet (GOOGL) performing financially recently, and are there any major news updates about them that might be relevant?"

response = research_agent.invoke(
    {'messages': [{'role': 'user', 'content': test_query1}]}
)

--- Tool Called: get_stock_price (Ticker: GOOGL) ------ Tool Called: get_company_info (Ticker: GOOGL) ---



In [8]:
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

How is Alphabet (GOOGL) performing financially recently, and are there any major news updates about them that might be relevant?
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_InJkFwMobpcYaDtocSycjT9J)
 Call ID: call_InJkFwMobpcYaDtocSycjT9J
  Args:
    ticker: GOOGL
  get_company_info (call_yKOqxoL9P6fEH7lRTTLuoktG)
 Call ID: call_yKOqxoL9P6fEH7lRTTLuoktG
  Args:
    ticker: GOOGL
  tavily_search_results (call_3IOEc0MmCC2FHPlHYdawLOWS)
 Call ID: call_3IOEc0MmCC2FHPlHYdawLOWS
  Args:
    query: Alphabet GOOGL news updates October 2023
================================= Tool Message =================================
Name: get_stock_price

The current stock price for GOOGL is $307.16.
================================= Tool Message =================================
Name: get_company_info

Company Information for Alphabet Inc. (GOOGL):
- Sec

# Creating Test Cases

The function defined below automates the process of running the agent with a query and packaging the interaction details into a format suitable for DeepEval (`LLMTestCase`).
1.  **Function Definition:**
    *   Takes `test_query` (the user's question), the `agent` itself, and `expected_tools` (a list of tools we anticipate the agent *should* call for this query) as input.
    *   Returns an `LLMTestCase` object.
2.  **Prepare Agent Input:**
    *   `inputs = {"messages": [HumanMessage(content=test_query)]}`: Formats the query into the structure the agent expects. `HumanMessage` represents input from a human user.
3.  **Invoke Agent:**
    *   `response = agent.invoke(inputs)`: Runs the agent with the query. The `response` contains the full sequence of messages exchanged during the agent's operation.
4.  **Extract Actual Output:**
    *   `actual_output = response['messages'][-1].content`: The agent's final answer is assumed to be the content of the last message in the sequence.
5.  **Extract Tool Call Details (Two-Pass Approach):** This is the core logic for understanding how the agent used its tools.
    *   **Initialization:** `tools_called_list`, `tool_call_invocations`, `tool_outputs_map` are initialized to store extracted information.
    *   **First Pass (Iterate through messages):**
        *   **Identify Tool Invocations (`AIMessage`):** When the LLM decides to use a tool, it emits an `AIMessage` containing `tool_calls`. Each `tool_call` has a `name`, `args` (parameters), and an `id`. This pass extracts these details and stores them in `tool_call_invocations`.
            *   `json.loads(tool_call['args'])`: Tool arguments are often JSON strings and need to be parsed into Python dictionaries.
        *   **Map Tool Outputs (`ToolMessage`):** When a tool executes, its result is returned to the agent in a `ToolMessage`. This message includes the `tool_call_id` (linking it back to the `AIMessage` that requested the tool) and the `content` (the tool's output). This pass populates `tool_outputs_map` to associate each tool call ID with its output.
    *   **Second Pass (Construct `ToolCall` objects):**
        *   Iterates through the `tool_call_invocations` collected in the first pass.
        *   For each invocation, it looks up the corresponding output from `tool_outputs_map` using the `id`.
        *   It then creates a `deepeval.test_case.ToolCall` object, which stores the tool's `name`, its `input_parameters`, and its `output`. These are added to `tools_called_list`.
6.  **Create `LLMTestCase`:**
    *   An `LLMTestCase` object is instantiated. This is the standard format DeepEval uses to represent a single test scenario.
    *   It's populated with:
        *   `input`: The original `test_query`.
        *   `actual_output`: The agent's final textual response.
        *   `tools_called`: The list of `ToolCall` objects detailing actual tool usage by the agent.
        *   `expected_tools`: The list of `ToolCall` objects passed into the function, indicating which tools *should* have been called. This is used by metrics like `ToolCorrectnessMetric`.

In [9]:
def create_test_case_from_query(test_query: str, agent, expected_tools) -> LLMTestCase:
    """
    Runs a query through a LangGraph agent, extracts interaction details,
    and formats them into a DeepEval LLMTestCase.

    Args:
        test_query: The natural language query to send to the agent.
        research_agent: The compiled LangGraph agent executor (result of create_react_agent).

    Returns:
        A DeepEval LLMTestCase object populated with the input, actual_output,
        and details of the tools called during the agent's execution.
    """
    # Prepare input for the agent
    inputs = {"messages": [HumanMessage(content=test_query)]}

    # Invoke the agent to get the full response including intermediate steps
    response = research_agent.invoke(inputs)
    messages = response['messages']

    actual_output = response['messages'][-1].content

    # --- Extract Tool Call Details ---
    tools_called_list: List[ToolCall] = [] # final list of DeepEval ToolCall objects
    tool_call_invocations = [] # Store (name, params_dict) tuples from AIMessage -> stores tool call requests
    tool_outputs_map = {} # Store {tool_call_id: output_content} from ToolMessage -> tool output text

    # First pass: Find tool invocations in AIMessages and map ToolMessage outputs
    for message in messages:
        if isinstance(message, AIMessage) and message.tool_calls:
            for tool_call in message.tool_calls:
                # Parameters are often JSON strings, parse them
                params_dict = json.loads(tool_call['args']) if isinstance(tool_call.get('args'), str) else tool_call.get('args', {})
                tool_call_invocations.append({
                    "id": tool_call.get('id'),
                    "name": tool_call['name'],
                    "input_parameters": params_dict
                })
        elif isinstance(message, ToolMessage) and message.tool_call_id:
            # Map output content to the specific tool_call_id it corresponds to
            tool_outputs_map[message.tool_call_id] = message.content

    # Second pass: Construct ToolCall objects using the mapped outputs
    if tool_call_invocations:
         for invocation in tool_call_invocations:
              tool_output = tool_outputs_map.get(invocation['id'], "[Output not found for this tool call ID]")

              tools_called_list.append(
                   ToolCall(
                        name=invocation['name'],
                        input_parameters=invocation['input_parameters'],
                        output=tool_output
                   )
              )

    # --- Create and Return LLMTestCase ---
    test_case = LLMTestCase(
        input=test_query,
        actual_output=actual_output,
        tools_called=tools_called_list,
        expected_tools=expected_tools
    )

    return test_case

# This function runs the agent on a test query and produces a DeepEval LLMTestCase containing the final answer.
# It also produces a structured log of tool calls (inputs + outputs), so you can evaluate tool usage and task completion.

We can now create specific test cases using the function defined above.

In the code block below, we define several specific test scenarios and uses the `create_test_case_from_query` function to generate DeepEval `LLMTestCase` objects for each.
1.  **Defining Queries:** Four distinct queries (`test_query1` was defined earlier, `test_query2`, `test_query3`, `test_query4`) are created. Each query is designed to test different aspects of the agent's capabilities:
    *   `test_query1`: Implied needs (financial performance suggests stock price, updates suggest news).
    *   `test_query2`: Specific company info and competitive news (tests `get_company_info` and general search).
    *   `test_query3`: Multiple requests for one company, including handling an invalid input (tests `get_stock_price` for valid and invalid tickers, and news).
    *   `test_query4`: Comparison requiring multiple tool calls for different entities and synthesis of information.
2.  **Setting Expected Tools:** For each test case, a list of `expected_tools` is provided. This list tells DeepEval which tools we anticipate the agent *should* ideally call to answer the query effectively.
    *   `ToolCall(name='get_stock_price')`: This indicates that we expect the `get_stock_price` tool to be called. The `ToolCorrectnessMetric` will use this information.
    *   Note: For `tavily_search_results`, it's often harder to specify it as an "expected tool" unless the query very explicitly demands a search. The examples focus on the custom financial tools. The `ToolCorrectnessMetric` in DeepEval checks if *all* tools listed in `expected_tools` were called, and if *only* tools that are relevant (which can be a broader set than just `expected_tools` if the agent uses others appropriately) were called.
3.  **Generating Test Cases:** `create_test_case_from_query` is called for each query, passing the query, the agent, and the corresponding `expected_tools`. The results (`test_case1`, `test_case2`, etc.) are `LLMTestCase` objects.
4.  **Aggregating Test Cases:**
    *   `test_cases = [test_case1, test_case2, test_case3, test_case4]`: All generated `LLMTestCase` objects are collected into a single list, `test_cases`, which will be fed into the DeepEval evaluation function.


In [10]:
test_case1 = create_test_case_from_query(
    test_query=test_query1,
    agent=research_agent,
    expected_tools=[ToolCall(name='get_stock_price'), ToolCall(name='get_company_info')]
)

--- Tool Called: get_stock_price (Ticker: GOOGL) ------ Tool Called: get_company_info (Ticker: GOOGL) ---



In [11]:
# Test Case 2: Specific comparison hint (Business Info + Competitive News)
test_query2 = "Give me a brief overview of Microsoft's business (ticker MSFT) and check for recent competitive news regarding their cloud offerings versus Google's."

test_case2 = create_test_case_from_query(
    test_query=test_query2,
    agent=research_agent,
    expected_tools=[ToolCall(name='get_company_info')]
)

--- Tool Called: get_company_info (Ticker: MSFT) ---


In [12]:
# Test Case 3: Focus on one company + Handling invalid input

test_query3 = "I need an update on Microsoft (MSFT). What's their current stock price and any significant news? Also, try to find the price for a ticker 'ABCFAKE'."

test_case3 = create_test_case_from_query(
    test_query=test_query3,
    agent=research_agent,
    expected_tools=[ToolCall(name='get_stock_price')]
)

--- Tool Called: get_stock_price (Ticker: MSFT) ---
--- Tool Called: get_stock_price (Ticker: ABCFAKE) ---
Error fetching stock price for ABCFAKE: HTTP Error 404: 


In [13]:
# Test Case 4: Comparison requiring multiple calls and synthesis

test_query4 = "Could you compare Microsoft (MSFT) and Alphabet (GOOGL)? I'm interested in their current market price and maybe a short summary of what each company does."

test_case4 = create_test_case_from_query(
    test_query=test_query4,
    agent=research_agent,
    expected_tools=[ToolCall(name='get_stock_price'), ToolCall(name='get_company_info')]
)

--- Tool Called: get_stock_price (Ticker: MSFT) ---
--- Tool Called: get_stock_price (Ticker: GOOGL) ---
--- Tool Called: get_company_info (Ticker: MSFT) ---
--- Tool Called: get_company_info (Ticker: GOOGL) ---


In [14]:
# Combine test cases
test_cases = [test_case1, test_case2, test_case3, test_case4]

# Defining Evaluation Metrics

The code block below defines the criteria (metrics) that will be used by DeepEval to evaluate the agent's performance on the test cases.
1.  **Task Completion Metric:**
    *   `task_completion_metric = TaskCompletionMetric(...)`: An instance of `TaskCompletionMetric` is created. This metric assesses how well the agent's final output (`actual_output` in the `LLMTestCase`) fulfills the user's request (`input` in `LLMTestCase`).
    *   `threshold=0.7`: Sets a passing threshold for this metric. If the metric's score (typically between 0 and 1) is 0.7 or higher, the test case is considered to have passed for this metric.
    *   `model='gpt-4o'`: Specifies that an LLM (here, `gpt-4o`, which should be a strong model like OpenAI's GPT-4o for reliable evaluation) should be used to judge task completion. This LLM compares the input query with the agent's output.
    *   `include_reason=True`: Instructs the metric to provide a textual explanation (reason) for its score, which is very helpful for understanding why a task was deemed complete or incomplete.
2.  **Tool Correctness Metric:**
    *   `tool_correctness_metric = ToolCorrectnessMetric()`: An instance of `ToolCorrectnessMetric` is created. This metric evaluates whether the agent used its tools appropriately. It typically considers:
        *   Were the `expected_tools` (from `LLMTestCase`) actually called?
        *   Were there any unnecessary or hallucinated tool calls?
        *   (Optionally, if parameters are specified in `expected_tools`) Were the parameters passed to the tools correct? (In this example, `expected_tools` only specify names, so parameter correctness isn't the primary focus here but the metric can be configured for it).

In [15]:
task_completion_metric = TaskCompletionMetric(
    threshold=0.7, # Setting a slightly higher bar
    model='gpt-4o',
    include_reason=True
)

tool_correctness_metric = ToolCorrectnessMetric()

# Running Evaluations

This is the final step where the actual evaluation takes place.
1.  **`evaluate(...)` function call:** The `evaluate` function from DeepEval is called.
    *   `test_cases=test_cases`: This argument provides the list of `LLMTestCase` objects that were prepared in Block 8. Each `LLMTestCase` contains the input query, the agent's actual output, details of tools called, and the expected tools.
    *   `metrics=[task_completion_metric, tool_correctness_metric]`: This argument provides the list of metric objects (defined in Block 9) that will be applied to each test case.
2.  **Evaluation Process:**
    *   DeepEval iterates through each `LLMTestCase` in the `test_cases` list.
    *   For each test case, it applies each metric in the `metrics` list.
    *   For example, the `TaskCompletionMetric` will use its configured LLM (`gpt-4o`) to compare the `input` (original query) with the `actual_output` (agent's final answer) from the `LLMTestCase` and generate a score and reason.
    *   The `ToolCorrectnessMetric` will compare the `tools_called` list with the `expected_tools` list from the `LLMTestCase` and generate a score.
3.  **Storing Results:**
    *   `results = ...`: The `evaluate` function returns a list of evaluation results. Each item in this list typically corresponds to a test case and contains the scores and reasons from all applied metrics for that test case. This `results` object can then be printed or further analyzed to understand the agent's performance.

In [16]:
results = evaluate(test_cases=test_cases, metrics=[task_completion_metric, tool_correctness_metric])

✨ You're running DeepEval's latest Task Completion Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Evaluating 4 test case(s) in parallel: |██████████|100% (4/4) [Time Taken: 00:04,  1.08s/test case]



Metrics Summary

  - ✅ Task Completion (score: 0.9, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The system successfully provided the current stock price and significant news for Microsoft, which was the primary goal. However, it failed to retrieve the stock price for 'ABCFAKE' due to it being an invalid ticker symbol, which slightly detracts from the overall goal achievement., error: None)
  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: All expected tools ['get_stock_price'] were called (order not considered)., error: None)

For test case:

  - input: I need an update on Microsoft (MSFT). What's their current stock price and any significant news? Also, try to find the price for a ticker 'ABCFAKE'.
  - actual output: ### Microsoft (MSFT) Update

- **Current Stock Price**: $485.92

#### Significant News:
1. **Microsoft Dragon Copilot**: Recently announced as the healthcare industry's first unified voice AI assistan

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.